In [29]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_blobs
from collections import OrderedDict
import torch.nn as nn
import pandas as pd
import torch

### Sequential with module and forward

In [30]:
def NeuralNetwork(Features, hidden, target):

	# construct a sequential neural network
	mlpModel = nn.Sequential(OrderedDict([
		("hidden_layer", nn.Linear(Features, hidden)),
		("activation_function", nn.Sigmoid()),
		("output_layer", nn.Linear(hidden, target)),
		("out", nn.Sigmoid())                                       # we use this to get values between 0 and 1
	]))
	
	return mlpModel

### Create Data

In [31]:
data = pd.read_csv('data.csv', header=None)
data

,0,1,2
0,0.78051,-0.063669,1
1,0.28774,0.291390,1
2,0.40714,0.178780,1
3,0.29230,0.421700,1
4,0.50922,0.352560,1
...,...,...,...
95,0.77029,0.701400,0
96,0.73156,0.717820,0
97,0.44556,0.579910,0
98,0.85275,0.859870,0


### Create X and y and split

In [32]:
# create x and y with tensor
X = torch.tensor(data.drop(2, axis=1).values, dtype= torch.float)
y = torch.tensor(data[2].values, dtype= torch.float).reshape(-1, 1)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.3)

In [34]:
X_train.shape

torch.Size([70, 2])

In [35]:
y_test.shape

torch.Size([30, 1])

### Train modle

In [36]:
torch.manual_seed(0)
model = NeuralNetwork(X.shape[1], 10, 1)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01)
loss_fn = nn.BCELoss()

In [37]:
losses = []                   
epochs = 500

# pass the data through the model for a number of epochs
for epoch in range(epochs):

    # put model in training model
    model.train()
    # forward pass on train data using the forward() method inside
    y_pred = model(X_train)
    # calculate the loss
    loss = loss_fn(y_pred, y_test)
    # zero the gradients of the optimizer
    optimizer.zero_grad()
    # perform backpropagation on the loss
    loss.backward()
    # progress the optimizer
    optimizer.step()

    # store loss 
    losses.append(loss.detach().item())
    print(f'Epoch {epochs}, loss {loss.item():.2f}')

ValueError: Using a target size (torch.Size([30, 1])) that is different to the input size (torch.Size([70, 1])) is deprecated. Please ensure they have the same size.